In [1]:
import pandas as pd
import os
import geopandas as gpd

In [2]:
path = os.path.dirname(os.getcwd())

In [3]:
#### Data of EAI with CI
data1 = pd.read_excel(os.path.join(path, 'eac_ci_df.xlsx'), index_col = 0)
data1 = data1.loc[:,0:2]
data1 = data1
data1

,0,1,2
0,25.408750,7.179986,5.495185
1,9.760083,3.525186,2.384388
2,7.379005,1.700874,0.976995


In [4]:
#### Data of EAI with CI
data2 = pd.read_excel(os.path.join(path, 'eac_noci_df.xlsx'), index_col = 0)
data2 = data2.loc[:,0:2]
data2 = data2
data2

,0,1,2
0,5.495185,5.495185,5.495185
1,2.384388,2.384388,2.384388
2,0.976995,0.976995,0.976995


In [5]:
data4 = (data1 - data2)/data1
data4

,0,1,2
0,0.783729,0.234652,0.0
1,0.755700,0.323613,0.0
2,0.867598,0.425593,0.0


In [6]:
data3 = (data1 / data2)
data3

,0,1,2
0,4.623820,1.306596,1.0
1,4.093328,1.478445,1.0
2,7.552758,1.740925,1.0


#### Fraction of failed substations

In [7]:
op = [1.0, 1.01, 1.025]
tf = [0, 0.25, 1]
RP = [10, 100, 1000, 10000]
cires = [0,1,2]

In [8]:
path_results = os.path.join(path, 'results')

In [9]:
damaged_ci = pd.DataFrame(index =  RP, columns = cires)

In [10]:
for i in range(len(cires)):
    for j in range(len(RP)):

        opf = op[0]
        tff = tf[0]
        rp = RP[j]
        ci = cires[i]
        
        ci_total = pd.read_excel(os.path.join(path_results, f'ci_{opf}_{tff}_mosek_{rp}_{ci}.xlsx'), index_col = [0])
        ci_failed = ci_total[ci_total['flood_fail'] == 0]
        ratio = len(ci_failed)
        damaged_ci.loc[rp, ci] = ratio

In [11]:
damaged_ci

,0,1,2
10,0,0,0
100,1,0,0
1000,7,2,0
10000,15,7,0


#### Fraction of industries without power to the flooded industries

In [12]:
path_data = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data')

In [13]:
osmsites = gpd.read_parquet(os.path.join(path_data, 'is_nl33.parquet'))
sectors = gpd.read_parquet(os.path.join(path_data, 'industrial.parquet'))

In [14]:
ind_ratio = pd.DataFrame(index =  RP, columns = cires)
fld_ratio = pd.DataFrame(index =  RP, columns = cires)

In [15]:
for i in range(len(cires)):
    for j in range(len(RP)):
        
        opf = op[0]
        tff = tf[0]
        rp = RP[j]
        ci = cires[i]

        os_df = pd.read_excel(os.path.join(path_results, f'osm_industrial_{1.01}_{0.25}_mosek_{rp}_{ci}.xlsx'), index_col = [0])
        
        # Assigning damage to geo dataframe
        osmsites['flood_fail'] = os_df['flood_fail']
        osmsites['power_fail'] = os_df['power_fail']
        
        
        sectors_centroids = sectors.copy()
        sectors_centroids['geometry'] = sectors_centroids.geometry.centroid
        
        sectors_in_osmsite = gpd.sjoin(
        sectors_centroids,
        osmsites[['geometry', 'osmid', 'flood_fail', 'power_fail']],
        how='inner',
        predicate='within'
        )
        
        sectors_in_osmsite = sectors_in_osmsite.reset_index(drop=True)
        flooded = sectors_in_osmsite[sectors_in_osmsite['flood_fail'] == 0]
        powerf = sectors_in_osmsite[(sectors_in_osmsite['power_fail'] == 0) & (sectors_in_osmsite['flood_fail'] == 1)]
        ratio = len(powerf)/ len(sectors_in_osmsite)
        ratio1 = len(flooded) / len(sectors_in_osmsite)
        ind_ratio.loc[rp,ci] = ratio
        fld_ratio.loc[rp,ci] = ratio1

C:\Users\sva100\AppData\Local\Temp\ipykernel_9436\1809345272.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sectors_centroids['geometry'] = sectors_centroids.geometry.centroid
C:\Users\sva100\AppData\Local\Temp\ipykernel_9436\1809345272.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sectors_centroids['geometry'] = sectors_centroids.geometry.centroid
C:\Users\sva100\AppData\Local\Temp\ipykernel_9436\1809345272.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sectors_centroids['geometry'] = sectors_centroids.geometry.centroid
C:\Users\sva100\AppData\Local\T

In [16]:
ind_ratio

,0,1,2
10,0.0,0.0,0.0
100,0.019907,0.0,0.0
1000,0.0929,0.038487,0.0
10000,0.198407,0.077638,0.0


In [17]:
data4.to_excel('eai_ratio.xlsx')
ind_ratio.to_excel('ind_ratio.xlsx')
damaged_ci.to_excel('damaged_ci.xlsx')
fld_ratio.to_excel('fld_ratio.xlsx')